In [108]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [109]:
#Extract Data From the PDF File

def load_pdf_file(data):
    loader= DirectoryLoader(data,
                                glob="*.pdf",
                                loader_cls=PyPDFLoader)
    
    documents=loader.load()

    return documents



In [110]:
extracted_data=load_pdf_file(data='Data/')

In [111]:
# extracted_data

In [112]:
#Split the data into text chunks

def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [113]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 273


In [114]:
#text_chunks

In [115]:
from langchain.embeddings import HuggingFaceEmbeddings

In [116]:
%pip install -U sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [117]:
# Download the Embeddings from Hugging Face

def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [118]:
embeddings = download_hugging_face_embeddings()

In [119]:
query_result = embeddings.embed_query("Hello World")
print("Length", len(query_result))

Length 384


In [120]:
#query_result

In [121]:
from dotenv import load_dotenv
load_dotenv()

True

In [122]:
%pip install python-dotenv

from dotenv import load_dotenv
import os

load_dotenv()  # This reads the .env file

PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

if not PINECONE_API_KEY:
    raise ValueError("PINECONE_API_KEY not found. Is your .env file loaded?")
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY not found. Is your .env file loaded?")


Note: you may need to restart the kernel to use updated packages.


In [123]:
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY


In [ ]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key= PINECONE_API_KEY)

index = pc.Index('students-score-padi')

In [125]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name="students-score-padi",
    embedding=embeddings,
)

In [126]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [127]:
retriever_docs = retriever.invoke("What is studying?")
retriever_docs

[Document(id='4dee3307-ab01-4bdf-bd74-e337677a1828', metadata={'author': 'Sarah Simpson', 'creationdate': '2015-10-12T00:00:00+00:00', 'creator': 'Bookboon.com Ltd.', 'moddate': '2019-07-10T16:45:50+00:00', 'page': 8.0, 'page_label': '9', 'producer': 'Adobe PDF Library 15.0; modified by SetaPDF-FormFiller Component v2.31.0.1330 ©Setasign 2005-2019 (www.setasign.com)', 'source': 'Data\\essential-study-skills.pdf', 'title': 'Essential Study Skills', 'total_pages': 86.0, 'trapped': '/False'}, page_content='Essential Study Skills\n9  \nIntroduction\n1 Introduction\nWe can describe study skills as “learning how to become a more effective learner”. They enable you to \nstudy and learn more efficiently, however, they do have to be practised and developed.\nDuring your lifetime of learning and study you will have to; learn, interpret, condense and recall huge \namounts of information. Clearly the specific skill, tool or technique you will use will be determined by;'),
 Document(id='4d3c840a-93

In [128]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [129]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = ("""
    You are an expert academic coach and learning strategist. Your task is to generate a personalized study guide for a student based on the topic they are studying. Your response should:

Use the following context to support your explanations:
{context}

1. Break down the topic into 3 to 5 clear sections or subtopics.
2. For each subtopic, explain it simply but deeply, as if teaching a beginner.
3. Include study advice for how to master that section (e.g., use flashcards, teach it to someone, spaced repetition).
4. Ask at least 2 **active recall questions** per subtopic to test the student’s understanding.
5. End the guide with a motivational note and a recap of the best study strategy for that topic.

Make the tone friendly, supportive, and focused on helping the student become exceptional in their understanding.

Topic: {topic}
Tailor the guide based on the student’s motivation level ({motivation_level}) and time available ({study_hours_per_week} hours/week). Give study strategies that fit their style ({preferred_learning_style}).

        """)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human","{input}"),
    ]
)


In [130]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [131]:
response = rag_chain.invoke({
    "input": "Can you create my personalized study guide?",
    "topic": "Calculus",
    "motivation_level": "Medium",
    "study_hours_per_week": "5",
    "preferred_learning_style": "Visual"
})


In [1]:
print(response)

NameError: name 'response' is not defined

In [132]:
print(response["answer"])



System: Of course! Let's get started on creating a personalized study guide for you on the topic of Calculus. Based on your motivation level (Medium) and time available (5 hours/week), I will tailor the guide to fit your needs. Since you prefer a visual learning style, I will include study strategies that will work best for you. 

Section 1: Introduction to Calculus
- In this section, you will learn the basic concepts and principles of Calculus, including limits, derivatives, and integrals.
- Study advice: Use flashcards to memorize key formulas and concepts. Teach the material to someone else to solidify your understanding.
- Active recall questions:
1. What is the definition of a limit?
2. How is a derivative calculated?

Section 2: Differentiation
- This section will cover the rules and techniques of differentiation, including the power rule, product rule, and chain rule.
- Study advice: Create visual aids such as diagrams and graphs to help you understand the concepts better. Use